In [66]:
import numpy as np
import cv2

In [67]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [68]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------
    # P=(x,y,z)->P'=(alpha*x/z+c_x,beta*y/z+c_y)
    alpha=K[0][0]
    beta=K[1][1]
    c_x=K[0][2]
    c_y=K[1][2]
    
    y_camera,x_camera=np.where(seg==255)
    depth=depth[seg==255]
   
    x_world=((x_camera-c_x)/alpha*depth).reshape(-1)
    y_world=((y_camera-c_y)/beta*depth).reshape(-1)
    z_world=depth.reshape(-1)

    pc=np.zeros((x_world.shape[0],3))
    pc[:,0]=x_world
    pc[:,1]=y_world
    pc[:,2]=z_world
    
    return pc

partial_pc = depth2pc(dpt, seg, K)
print(partial_pc)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

[[-0.09423124 -0.40291978  1.3507307 ]
 [-0.09079667 -0.40209955  1.34798099]
 [-0.08738345 -0.4013166   1.34535627]
 ...
 [ 0.32885786  0.24503135  1.34023182]
 [ 0.33220583  0.24512275  1.34073177]
 [ 0.33586891  0.24544267  1.34248158]]


In [69]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------
one_way_CD = 0
print(partial_pc_sampled.shape)
t1=np.tile(partial_pc_sampled,(2048,1))
t2=np.repeat(full_pc_sampled,2048,axis=0)
t3=np.sqrt(np.sum((t1-t2)**2,axis=1)).reshape(2048,2048)
one_way_CD=[np.mean(np.min(t3,axis=0))]
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD)

(2048, 3)
one way chamfer distance:  [0.010298205005798703]
